In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import hockey_rink
import hockey_scraper
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math

Gonna get familiar with hockey_scraper, never used it until now. Scraping the schedule between 2023-09-01 and 2024-08-31. Should be 1,312 NHL games in a season.

In [ ]:
hockey_scraper.scrape_seasons([2023,2024], True)

Noe have shift data for the nhl 2023-24 season as well as the play by play data (pbp) in separate csv files. 

In [16]:
#Loading in pbp data
pbp=pd.read_csv("nhl_pbp_20232024.csv")

In [17]:
#Loading in shift data
shifts=pd.read_csv("nhl_shifts_20232024.csv")

Step 1: EDA

In [18]:
shifts.head(5)

,Unnamed: 0,Game_Id,Period,Team,Player,Player_Id,Start,End,Duration,Date
0,0,20001,1,NSH,RYAN MCDONAGH,8474151.0,0.0,30.0,30.0,2023-10-10
1,1,20001,1,NSH,RYAN O'REILLY,8475158.0,0.0,35.0,35.0,2023-10-10
2,2,20001,1,T.B,VICTOR HEDMAN,8475167.0,0.0,41.0,41.0,2023-10-10
3,3,20001,1,NSH,FILIP FORSBERG,8476887.0,0.0,32.0,32.0,2023-10-10
4,4,20001,1,T.B,TYLER MOTTE,8477353.0,0.0,32.0,32.0,2023-10-10


In [19]:
pbp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445637 entries, 0 to 445636
Data columns (total 57 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       445637 non-null  int64  
 1   Game_Id          445637 non-null  int64  
 2   Date             445637 non-null  object 
 3   Period           445637 non-null  int64  
 4   Event            445637 non-null  object 
 5   Description      445637 non-null  object 
 6   Time_Elapsed     445637 non-null  object 
 7   Seconds_Elapsed  445637 non-null  float64
 8   Strength         445637 non-null  object 
 9   Ev_Zone          369193 non-null  object 
 10  Type             180765 non-null  object 
 11  Ev_Team          369085 non-null  object 
 12  Home_Zone        369193 non-null  object 
 13  Away_Team        445637 non-null  object 
 14  Home_Team        445637 non-null  object 
 15  p1_name          369011 non-null  object 
 16  p1_ID            360845 non-null  floa

See the values inside home goalie and away goalie. Trying to figure out how to determine if net is empty


In [23]:
pbp['Home_Goalie'].unique()
#nan represents no goalie

array(['JONAS JOHANSSON', nan, 'TRISTAN JARRY', 'ADIN HILL',
       'FREDERIK ANDERSEN', 'ILYA SAMSONOV', 'LINUS ULLMARK',
       'JACOB MARKSTROM', 'CAM TALBOT', 'THATCHER DEMKO', 'CASEY DESMITH',
       'DEVON LEVI', 'ELVIS MERZLIKINS', 'VITEK VANECEK',
       'JAKE OETTINGER', 'FILIP GUSTAVSSON', 'JUUSE SAROS',
       'KAAPO KAHKONEN', 'AKIRA SCHMID', 'CHARLIE LINDGREN',
       'ANTON FORSBERG', 'CONNOR HELLEBUYCK', 'JEREMY SWAYMAN',
       'SPENCER MARTIN', 'VILLE HUSSO', 'SAM MONTEMBEAULT',
       'ALEX NEDELJKOVIC', 'ILYA SOROKIN', 'JORDAN BINNINGTON',
       'STUART SKINNER', 'MACKENZIE BLACKWOOD', 'PHEONIX COPLEY',
       'JOONAS KORPISALO', 'LUKAS DOSTAL', 'IGOR SHESTERKIN',
       'JOSEPH WOLL', 'DARCY KUEMPER', 'CARTER HART', 'KEVIN LANKINEN',
       'PHILIPP GRUBAUER', 'SERGEI BOBROVSKY', 'JONATHAN QUICK',
       'MARC-ANDRE FLEURY', 'JOEL HOFER', 'LAURENT BROSSOIT',
       'JOHN GIBSON', 'JOEY DACCORD', 'ALEXANDAR GEORGIEV',
       'KAREL VEJMELKA', 'ERIC COMRIE', 'JAKE AL

In [27]:
pbp[(pbp['Home_Goalie'].isna()) & (pbp['Period']==3)]

,Unnamed: 0,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,...,Away_Score,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach
206,206,20001,2023-10-10,3,PSTR,Period Start- Local time: 7:21 EDT,0:00,0.0,-1x-1,NaN,...,1,1,NaN,NaN,NaN,NaN,-69.0,-17.0,Jon Cooper,Andrew Brunette
234,234,20001,2023-10-10,3,GOAL,"TBL #38 HAGEL(1), Penalty Shot, Wrist , Off. Z...",3:07,187.0,0x0,Off,...,2,2,JUUSE SAROS,8477424.0,NaN,NaN,45.0,-2.0,Jon Cooper,Andrew Brunette
327,327,20001,2023-10-10,3,GEND,Game End- Local time: 8:05 EDT,20:00,1200.0,-1x-1,NaN,...,3,5,NaN,NaN,NaN,NaN,-86.0,-37.0,Jon Cooper,Andrew Brunette
644,644,20002,2023-10-10,3,FAC,PIT won Off. Zone - CHI #98 BEDARD vs PIT #87 ...,17:25,1045.0,5x5,Off,...,3,2,PETR MRAZEK,8475852.0,NaN,NaN,NaN,NaN,Mike Sullivan,Luke Richardson
645,645,20002,2023-10-10,3,SHOT,"PIT ONGOAL - #71 MALKIN, Slap , Off. Zone, 58 ...",17:28,1048.0,5x5,Off,...,3,2,PETR MRAZEK,8475852.0,NaN,NaN,36.0,-24.0,Mike Sullivan,Luke Richardson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444965,444965,30415,2024-06-18,3,GEND,Game End- Local time: 10:54 EDT,20:00,1200.0,-1x-1,NaN,...,5,3,NaN,NaN,NaN,NaN,10.0,36.0,Paul Maurice,Kris Knoblauch
445196,445196,30416,2024-06-21,3,PSTR,Period Start- Local time: 8:18 MDT,0:00,0.0,-1x-1,NaN,...,0,3,NaN,NaN,NaN,NaN,NaN,NaN,Kris Knoblauch,Paul Maurice
445305,445305,30416,2024-06-21,3,GEND,Game End- Local time: 8:57 MDT,20:00,1200.0,-1x-1,NaN,...,1,5,NaN,NaN,NaN,NaN,NaN,NaN,Kris Knoblauch,Paul Maurice
445522,445522,30417,2024-06-24,3,PSTR,Period Start- Local time: 10:12 EDT,0:00,0.0,-1x-1,NaN,...,1,2,NaN,NaN,NaN,NaN,NaN,NaN,Paul Maurice,Kris Knoblauch


Step 2: Data Cleansing

Definitely do not need many of these variables so going to drop a bunch of them

In [6]:
pbp=pbp.drop(['Away_Coach','Home_Coach','Home_Goalie_Id','Home_Goalie','Away_Goalie_Id','Away_Goalie'], axis=1)

In [7]:
pbp.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
pbp[['awayPlayer1','awayPlayer2', 'awayPlayer3','awayPlayer4','awayPlayer5','awayPlayer6']].head(10)

,awayPlayer1,awayPlayer2,awayPlayer3,awayPlayer4,awayPlayer5,awayPlayer6
0,JUUSO PARSSINEN,RYAN O'REILLY,FILIP FORSBERG,RYAN MCDONAGH,ALEXANDRE CARRIER,JUUSE SAROS
1,JUUSO PARSSINEN,RYAN O'REILLY,FILIP FORSBERG,RYAN MCDONAGH,ALEXANDRE CARRIER,JUUSE SAROS
2,YAKOV TRENIN,RYAN O'REILLY,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
3,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
4,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
5,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
6,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
7,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
8,CODY GLASS,GUSTAV NYQUIST,KIEFER SHERWOOD,JEREMY LAUZON,TYSON BARRIE,JUUSE SAROS
9,CODY GLASS,GUSTAV NYQUIST,KIEFER SHERWOOD,JEREMY LAUZON,TYSON BARRIE,JUUSE SAROS


Alright looks like players 1,2, and 3 are the forwards. 4 and 5 are the defensemen and 6 is the goalie, So i only need players 1-3

In [9]:
pbp=pbp.drop(['awayPlayer4','awayPlayer4_id','awayPlayer5','awayPlayer5_id','awayPlayer6','awayPlayer6_id','homePlayer4','homePlayer4_id','homePlayer5','homePlayer5_id','homePlayer6','homePlayer6_id'], axis=1)

Got rid of a ton of columns not needed. All columns now should be useful

In [10]:
pbp.columns

Index(['Game_Id', 'Date', 'Period', 'Event', 'Description', 'Time_Elapsed',
       'Seconds_Elapsed', 'Strength', 'Ev_Zone', 'Type', 'Ev_Team',
       'Home_Zone', 'Away_Team', 'Home_Team', 'p1_name', 'p1_ID', 'p2_name',
       'p2_ID', 'p3_name', 'p3_ID', 'awayPlayer1', 'awayPlayer1_id',
       'awayPlayer2', 'awayPlayer2_id', 'awayPlayer3', 'awayPlayer3_id',
       'homePlayer1', 'homePlayer1_id', 'homePlayer2', 'homePlayer2_id',
       'homePlayer3', 'homePlayer3_id', 'Away_Players', 'Home_Players',
       'Away_Score', 'Home_Score', 'xC', 'yC'],
      dtype='object')

Variables we want, many of which need to be modified or created:
    1)xC

    2)yC

    3)Type

    4)shotDistance (Create)

    5)shotAngle (Create)

    6)timeSinceLastEvent (Create)

    7)distanceFromLastEvent (Create)

    8)PowerPlay (Create)
    
    9)speedSinceLastEvent (Create)

Making all shots occur on the right side of the rink

In [30]:
#First making all shots appear on the positive (right side of the rink)
pbp['xC']=np.where(pbp['xC']<0, pbp['xC']*-1, pbp['xC'])
pbp['yC']=np.where(pbp['xC']<0, pbp['yC']*-1, pbp['yC'])

Getting shot distance from x and y coordinates

In [31]:
pbp['shotDistance']= np.sqrt((pbp['yC'] - 0)**2 + (pbp['xC'] - 89.0)**2)

Getting shot angle in degrees to the center of the net at (89,0)

In [33]:
pbp['Radians']=np.arcsin(pbp['shotDistance'])
pbp['shotAngle']=(pbp['Radians'] * 180) / 3.14


invalid value encountered in arcsin


In [34]:
pbp[['shotDistance','shotAngle','xC','yC']].head(5)

,shotDistance,shotAngle,xC,yC
0,NaN,NaN,NaN,NaN
1,48.877398,NaN,64.0,42.0
2,60.959003,NaN,43.0,-40.0
3,NaN,NaN,NaN,NaN
4,93.037627,NaN,5.0,-40.0


Alright, now that we got rid of a good number of columns we don't need, let's think about the problem at hand. I want to evaluate who are the best 20 forward lines in the nhl. How should I accomplish this? I think the best way from a hollistic standpoint is expected goals %. Expected Goals % is the percent of all expected goals the player's team gets while the player is on the ice compared to the other team. For example, if a player's team gets 0.6 expected goals and the opposing team gets 0.4 expected goals while the player is on the ice, the player's expected goals % is 60%. 

On MoneyPuck, an xGoals% of above 60% gets a forward line inside the top 10 (Limiting to lines who played 200 minutes or more together).An xGoals% of 50% and above gets a FL inside the top 60 lines. 

I also want to look at each lines expected goals against and their expected goals for, rather than the percentage of expected goal they own. An expected goals for of 28 or above gets you in the top 10, and an xG For of 20 gets you inside the top 25. 

For xG against, 8.5 or less gets you inside the top 10, 11 or less gets you inside the top 25. 

By looking at these xG for and xG against rather than xG%, I can get the best offensive forward lines and the best defensive lines.

Thus, at the very end of this project my goal is to have 3 different rankings:
1) The best 15 lines in hockey (using xG%)
2) The best 15 offensive lines in hockey (using xG For)
3) The best 15 defensive lines in hockey (using xG Against)

In [12]:
pbp['Event'].value_counts()

Event
FAC       79530
SHOT      76148
HIT       66216
STOP      62065
BLOCK     48100
MISS      38216
GIVE      21825
TAKE      19603
PENL      10675
GOAL       8772
PSTR       4574
PEND       4574
DELPEN     3605
GEND       1400
CHL         248
SOC          82
EISTR         2
EIEND         2
Name: count, dtype: int64

In [13]:
pbp['Ev_Zone'].value_counts()

Ev_Zone
Off    194813
Def    130455
Neu     43925
Name: count, dtype: int64

In [15]:
pbp['Type'].value_counts()

Type
WRIST SHOT                              99802
SNAP SHOT                               23664
SLAP SHOT                               20758
TIP-IN                                  11837
BACKHAND                                10216
                                        ...  
OS #61 MAROON High-sticking(2 min)          1
YR #96 ROSLOVIC Tripping(2 min)             1
AL #3 TANEV Delay Game(2 min)               1
YR #96 ROSLOVIC High-sticking(2 min)        1
AL #28 PETROVIC Tripping(2 min)             1
Name: count, Length: 217, dtype: int64

Step 3: Building the model and features to be passed into the model

Alright, in the expected goals model I am building, these variables (have to create a lot) are important:
    1)xC
    2)yC
    3)Type
    4)shotDistance (Create)
    5)shotAngle (Create)
    6)timeSinceLastEvent (Create)
    7)distanceFromLastEvent (Create)
    8)PowerPlay (Create)
    9)speedSinceLastEvent (Create)

Once Model is built, then since we are trying to find best forward lines, Group by player1, player2, player3. Sum their expected goals. Take top 15.